### Utilizando o dataset iris, muito conhecido, será feito o treinamento para prever qual tipo de espécie de flor

- Iris Setosa
- Iris Versicolour
- Iris Virginica

### E será selecionada com base em:

- largura da sépula em cm.
- altura da sépula em cm.
- largura da pétala em cm.
- altura da pétala em cm.


In [ ]:
from sklearn import datasets


iris_dataset = datasets.load_iris()
x = iris_dataset.data
""" A entrada do dataset """

x


In [ ]:
y = iris_dataset.target
""" A saída do dataset(as espécies de flor) """

y


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)
""" o array de entrada escalonado """

x


In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()

y.shape


In [ ]:
y = y.reshape(-1, 1)
""" a matriz de resposta  """
y.shape


In [ ]:
y = one_hot_encoder.fit_transform(y).toarray()

y


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42)


In [ ]:
x_train.shape


In [ ]:
x_test.shape


In [ ]:
input_neurons = x.shape[1]
""" os neurônios de entrada(4), (largura e altura da sépala e da pétala) """

input_neurons


In [ ]:
import numpy as np

ouput_hidden_neurons = int(np.ceil((x.shape[1] + y.shape[1])/2))

ouput_hidden_neurons


In [ ]:
output_neurons = y.shape[1]
""" As três classes de saída, Iris Setosa, Iris Versicolour e Iris Virginica """


In [ ]:
import tensorflow as tf

weights = {
    'hidden': tf.Variable(tf.random.normal([input_neurons, ouput_hidden_neurons]), name='hidden_weights'),
    'output': tf.Variable(tf.random.normal([ouput_hidden_neurons, output_neurons]), name='output_weights')
}


In [ ]:
bias = {
    'hidden': tf.Variable(tf.random.normal([ouput_hidden_neurons]), name='hidden_bias'),
    'output': tf.Variable(tf.random.normal([output_neurons]), name='output_bias')
}


In [ ]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32, name='x_train')
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32, name='y_train')
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32, name='x_test')
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32, name='y_test')


In [ ]:
def mlp(x, weights, bias):
    hidden_layer = tf.add(tf.matmul(x, weights['hidden']), bias['hidden'])
    hidden_layer_activation = tf.nn.relu(hidden_layer)

    output_layer = tf.add(
        tf.matmul(hidden_layer_activation, weights['output']), bias['output'])

    return output_layer


In [ ]:
with tf.GradientTape() as tape:
    model = mlp(x_train, weights, bias)
    loss = tf.nn.softmax_cross_entropy_with_logits(
        labels=y_train, logits=model)

var_list = [
  weights['hidden'], 
  weights['output'], 
  bias['hidden'], 
  bias['output']
  ]

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

gradients = tape.gradient(loss, var_list)

optimizer.apply_gradients(zip(gradients, var_list))

In [ ]:
batch_size = 8
batch_total = int(len(x_train) / batch_size)

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

dataset

In [ ]:
for epoch in range(3000):
    for x_batch, y_batch in dataset:
        standard_error = 0.0
        with tf.GradientTape() as tape:
            model = mlp(x_batch, weights, bias)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                labels=y_batch, logits=model))
        gradients = tape.gradient(loss, var_list)
        optimizer.apply_gradients(zip(gradients, var_list))
        standard_error += loss.numpy()
    if epoch % 50 == 0:
        print('Época: ' + str((epoch)) + ' erro: ' + str(standard_error))


In [ ]:
from sklearn.metrics import accuracy_score

predict_test = mlp(x_test, weights, bias)
predict_test = tf.nn.softmax(predict_test)
predict_test = tf.argmax(predict_test, axis=1).numpy()
print(accuracy_score(tf.argmax(y_test, axis=1).numpy(), predict_test))
